In [8]:
import yfinance as yf
import pandas as pd
import matplotlib as plt
import numpy
import datetime

In [62]:
djia_constituents = pd.read_csv(r"C:\Users\malani\Documents\Trading_Bot\Stocks in the Dow Jones Industrial Average.csv")
djia_constituents = djia_constituents.drop(25)
djia_constituents.tail()

,Symbol,Description,Category2,GICS Sector,Market cap,Market Cap Weight,Index Weight,Dividend yield,Country,Action
24,KO,Coca-Cola,Common stocks,Consumer Staples,"$262,541,200,257",2.60%,1.19%,2.84%,USA,Analyze
26,CSCO,Cisco Systems Inc,Common stocks,Information Technology,"$191,431,049,741",1.90%,0.89%,3.21%,USA,Analyze
27,WBA,Walgreen Boots Alliance Inc,Common stocks,Consumer Staples,"$34,704,949,342",0.34%,0.75%,4.69%,USA,Analyze
28,VZ,Verizon Communications Inc,Common stocks,Communication Services,"$160,265,033,778",1.59%,0.75%,6.60%,USA,Analyze
29,INTC,Intel Corporation Corp,Common stocks,Information Technology,"$123,356,030,000",1.22%,0.58%,4.80%,USA,Analyze


In [68]:
# sample = pd.DataFrame(columns=['Ticker', 'Prices'])
sample = {}

for i in range(len(djia_constituents)):
    ticker=yf.Ticker(djia_constituents.iat[i, 0])
    data = ticker.history(period='1846d')
    del data['Dividends']
    del data['Stock Splits']
    del data['Open']
    del data['High']
    del data['Low']
    del data['Volume']
    data["SMA"] = data[["Close"]].rolling(20).mean()
    data["STD"] = data[["Close"]].rolling(20).std()
    data = data.tail(data.shape[0] -20)

    # sample.insert(i, "Ticker", ticker)
    sample.update({djia_constituents.iat[i, 0] : data})

print(sample['AAPL'])

                                Close         SMA       STD
Date                                                       
2015-08-21 00:00:00-04:00   24.102167   26.598166  0.938804
2015-08-24 00:00:00-04:00   23.500528   26.380563  1.119692
2015-08-25 00:00:00-04:00   23.641815   26.163105  1.209220
2015-08-26 00:00:00-04:00   24.997795   26.017870  1.162860
2015-08-27 00:00:00-04:00   25.733892   25.916472  1.088848
...                               ...         ...       ...
2022-11-14 00:00:00-05:00  148.279999  145.559690  5.549215
2022-11-15 00:00:00-05:00  150.039993  145.886093  5.614038
2022-11-16 00:00:00-05:00  148.789993  146.144505  5.623264
2022-11-17 00:00:00-05:00  150.720001  146.522879  5.665783
2022-11-18 00:00:00-05:00  151.289993  146.736073  5.765067

[1826 rows x 3 columns]
